# Primer scrapeo

El proceso de scrapear una página web es parecido a lo que hace un humano cuando quiere buscar algo en Internet, la diferencia es que en lugar de ver el contenido presentado por el navegador, el programa analiza y selecciona el código fuente generalmente programado en  HTML y JavaScript.

El primer paso es por tanto seleccionar una página web para hacer el scraping y descargarla. Ya sabemos cómo hacer peticiones HTTP mediante `requests` así que vamos a a hacer una petición de la web de ejemplo por excelencia:

In [24]:
import requests

req = requests.get("https://www.uax.com")

El caso es que como respuesta a la petición se nos ha develto la página y podemos ver su código fuente en crudo:

In [25]:
#Todo el html asociado a la página
print(req.text)

<!doctype html>
<html lang="es">
  <head>
    <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
    <meta content="telephone=no" name="format-detection"/>
    <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>

          
    
                            <link rel="alternate" href="https://www.uax.com/en" hreflang="en" />
                            <link rel="alternate" href="https://www.uax.com/" hreflang="es" />
                            <link rel="alternate" href="https://www.uax.com/fr" hreflang="fr-FR" />
                            <link rel="alternate" href="https://www.uax.com/it" hreflang="it-IT" />
    
    
                      
                    
    <title>Universidad Alfonso X El Sabio: Universidad Privada en Madrid - UAX</title>

    <meta http-equiv="Content-Type" content="text/html; charset=utf-8"/>
    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"/>
    <meta name="viewport" content="width=device-width, initial-

Aquí tenemos un documento HTML bien estructurado con sus etiquetas.

Cuando el navegador interpreta estas etiquetas que se abren y se cierran, con sus atributos y contenidos genera lo que se conoce como **DOM** (*Document Object Model*), una interfaz de programación para documentos HTML y XML que en esencia es como un árbol ramificado de  componentes padres e hijos. El padre de todo es `html`, que tiene dos hijos `head` y `body`, el primero contiene el `title` y los metadatos, el otro el contenido de la página, una capa `div` que a su vez tiene una cabecera `h1` y unos parágrados `p`.

Pues bien, la biblioteca `BeautifulSoup` lo que hace es generar su propia estructura parecida a la interfaz **DOM** pero en Python, creando un árbol con los elementos del documento. Básicamente le pasamos un documento HTML en crudo y ella lo transforma en un objeto dinámico con el que podemos interactuar:

In [26]:
#El DOM de python

from bs4 import BeautifulSoup

soup = BeautifulSoup(req.text)


print(soup)

<!DOCTYPE html>
<html lang="es">
<head>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="telephone=no" name="format-detection"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<link href="https://www.uax.com/en" hreflang="en" rel="alternate"/>
<link href="https://www.uax.com/" hreflang="es" rel="alternate"/>
<link href="https://www.uax.com/fr" hreflang="fr-FR" rel="alternate"/>
<link href="https://www.uax.com/it" hreflang="it-IT" rel="alternate"/>
<title>Universidad Alfonso X El Sabio: Universidad Privada en Madrid - UAX</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="_FxizosYHI1BXMZq2zf9KJEPr6ktFvp3rw-_lzXkcCY" name="google-site-verification"/>
<link href="https://www.uax.com/" rel="canonical"/>
<meta content="Universidad Alfonso X El Sabio: Universidad 

A simple vista parece lo mismo, pero ahora podemos hacer algo como esto para consultar el título de la página:

In [27]:
#Seleccionar por el la etiqueta y devuelve el html
soup.select("title")

[<title>Universidad Alfonso X El Sabio: Universidad Privada en Madrid - UAX</title>]

Esto que nos devuelve es un objeto, veamos tu tipo:

In [28]:
#Selecciona la etiqueta y devuelve el tipo de objeto
type(soup.select("title"))

bs4.element.ResultSet

Es un conjunto de resultados que contiene los tags que concuerdan con el nombre `title`, por tanto es una lista.

Veamos qué tipo tiene ese primer valor del conjunto:

In [29]:
#De la lista, selecciona la primera
type(soup.select("title")[0])

bs4.element.Tag

Como véis es un `Tag` y éste contiene diferentes métodos, como por ejemplo `getText()` para recuperar su contenido:

In [30]:
#Selecciona la lista y el primer elemento para obtener su texto asociado
soup.select("title")[0].getText()

'Universidad Alfonso X El Sabio: Universidad Privada en Madrid - UAX'

Podemos recuperar otros elementos esenciales como la cabecera o los parágrafos:

In [31]:
#Selecciona todas las etiquetas de un mismo estilo
soup.select("h1")

[<h1 class="hero__content__title--sml hero__content__title" style="color: white;">Visita UAX de Lunes a Sábado</h1>,
 <h1 class="banner-secondary__title t-h1">Descubre todas nuestras titulaciones en online y presencial</h1>,
 <h1 class="banner-secondary__title t-h1">Bienvenido a UAX Rafa Nadal School of Sport</h1>]

In [32]:
#Primer elemento del tipo de etiqueta a
soup.select("a")[0]

<a class="header-mobile__logo" href="/" title=" info.uax.brand_name">
<img alt=" info.uax.brand_name" src="/assets/uax/images/logo.svg"/>
</a>

Fijaros que el segundo parágrafo contiene a su vez un enlace, podemos acceder de forma anidada:

In [33]:
soup.select("p")[0]

<p class="header__submenu-title">Por tipo de programa</p>

In [34]:
# Seleccionar del segundo parágrafo el primer enlace

a = soup.select("p")[0]

# Mostrar su contenido
a.getText()

'Por tipo de programa'

Las etiquetas tienen valores especiales llamados atributos, como la dirección `href` de un enlace. 

Estos se almacenan como un diccionario del objeto, es muy cómodo acceder a ellos:

In [35]:
# Atributo con la dirección del enlace
a=soup.select("a")[0]
#Para sonseguir uno de los atributos asociados a una etiqueta
a['href']

'/'

Estos valores están mapeados del diccionario `attrs`:

In [36]:
#pseudodiccionario con los atributos y sus valores
a.attrs.items()

dict_items([('class', ['header-mobile__logo']), ('href', '/'), ('title', ' info.uax.brand_name')])

Siguiendo esta lógica podemos programar un script que recupere todos los atributos de los metadatos:

In [37]:
for meta in soup.select("meta"):
    for atributo, valor in meta.attrs.items():
        print(f"{atributo}: {valor}")

content: width=device-width, initial-scale=1.0
name: viewport
content: telephone=no
name: format-detection
content: IE=edge,chrome=1
http-equiv: X-UA-Compatible
http-equiv: Content-Type
content: text/html; charset=utf-8
http-equiv: X-UA-Compatible
content: IE=edge,chrome=1
name: viewport
content: width=device-width, initial-scale=1.0
name: google-site-verification
content: _FxizosYHI1BXMZq2zf9KJEPr6ktFvp3rw-_lzXkcCY
property: og:title
content: Universidad Alfonso X El Sabio: Universidad Privada en Madrid - UAX
property: og:image
content: https://www.uax.com/assets/uax/images/logo-share.png
name: twitter:card
content: summary
name: twitter:title
content: Universidad Alfonso X El Sabio: Universidad Privada en Madrid - UAX
name: twitter:image
content: https://www.uax.com/assets/uax/images/logo-share.png
name: description
content: La Universidad Alfonso X el Sabio - UAX (Madrid) es la primera universidad privada española. Consúltanos en 91 810 92 00 o en info@uax.es
property: og:descriptio

Solo con esto os podéis hacer una ideal del potencial que tiene.